<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Notebooks/RoleMappingAspire/FullAirtableJune25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook-Beschreibung: Analyse von Jobrollen und Funktionsgruppen

Dieses Notebook führt eine umfassende Analyse von Jobrollen in verschiedenen Funktionseinheiten (`HOME FUNCTION`) durch und verwendet Daten aus Google Sheets, um verschiedene Perspektiven auf die Rollenverteilung zu bieten. Ziel ist es, Einblicke in die Zuordnung von Rollen zu Personen und deren Verteilung über die Organisation hinweg zu gewinnen.

**Die wichtigsten Schritte und Analysen, die in diesem Notebook durchgeführt werden, sind:**

1.  **Daten laden und vorbereiten:**
    *   Es werden Daten aus zwei Google Sheets geladen: die Hauptliste (`FullList`) mit allen Personen- und Rolleninformationen und eine Filterliste (`FilterRoles`), die definiert, welche Jobrollen für die Analyse relevant sind.
    *   Nur Jobrollen, die in der `FilterRoles`-Tabelle explizit als 'Relevant' markiert sind, werden für alle weiteren Analysen berücksichtigt. Dies stellt sicher, dass nur aussagekräftige Daten verarbeitet werden.
    *   Jobrollen, die als kommagetrennte Liste in einer Zelle vorliegen, werden aufgeteilt, sodass jede einzelne Rolle einer Person zugeordnet werden kann. Dadurch kann jede Rolle individuell analysiert werden.

2.  **Übersichten zur Rollenverteilung nach 'HOME FUNCTION':**
    *   **Pivot-Tabelle der Rollenanzahl pro Funktion (`job_role_summary_by_function.csv`):** Eine Übersicht, die anzeigt, wie viele Personen welche relevante Jobrolle in den einzelnen 'HOME FUNCTION'-Gruppen innehaben. Das Ergebnis wird als CSV-Datei exportiert.
    *   **Aggregierte Liste im Airtable-Format (`job_role_summary_airtable_format.csv`):** Eine vereinfachte Liste, die jede einzigartige Kombination aus 'HOME FUNCTION' und 'Job Role' auflistet, zusammen mit der Anzahl der Personen, die diese spezifische Rolle in dieser Funktionseinheit haben. Auch hier wird das Ergebnis als CSV exportiert.
    *   **Rollen-Funktions-Matrix mit Nullen (`job_role_function_matrix.csv`):** Eine detaillierte Pivot-Tabelle, die alle relevanten Jobrollen (Zeilen) und alle 'HOME FUNCTION'-Gruppen (Spalten) darstellt. Sie zeigt die Anzahl der Personen für jede Kombination und füllt fehlende Werte mit 0 auf, um ein vollständiges Bild zu geben. Das Ergebnis wird ebenfalls als CSV exportiert.

3.  **Personenbezogene Rollenaggregation (`person_role_filtered_aggregated.csv`):**
    *   Eine Liste wird erstellt, die jede Person mit ihren zugeordneten, **relevanten** Jobrollen anzeigt. Wenn eine Person mehrere relevante Rollen hat, werden diese in einer kommagetrennten Liste zusammengefasst.
    *   Diese Liste ist systematisch sortiert: zuerst nach 'HOME FUNCTION' und dann alphabetisch nach dem vollständigen Namen der Mitarbeiter (`EMPLOYEE FULL NAME`), um eine klare Struktur zu gewährleisten. Das Ergebnis wird als CSV exportiert.

4.  **Spezifische Analyse der Rolle "1201_General Assets Display":**
    *   Es wird untersucht, wie viele Personen ausschließlich die Rolle "1201_General Assets Display" haben, im Vergleich zu jenen, die diese Rolle **zusammen** mit mindestens einer weiteren relevanten Rolle besitzen.
    *   Diese Verteilung wird zur besseren Visualisierung in einem **Kuchendiagramm** dargestellt.

5.  **Verteilung der Anzahl relevanter Rollen pro Person:**
    *   Ein **Histogramm** visualisiert die Häufigkeit, mit der Personen eine bestimmte Anzahl von relevanten Rollen haben (begrenzt auf 1 bis 30 Rollen).
    *   Personen, die mehr als 30 relevante Rollen besitzen, werden separat in einer Tabelle zusammengefasst und die Gesamtanzahl der betroffenen Personen wird ausgegeben.

Dieses Notebook bietet somit einen tiefgehenden Einblick in die Struktur der Jobrollenverteilung innerhalb der Organisation, sowohl auf aggregierter Ebene als auch detailliert pro Mitarbeiter.

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren für Google Drive & Sheets
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen über URL
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8/edit?gid=647371453#gid=647371453"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Reiter "FullList" einlesen als DataFrame
worksheet = spreadsheet.worksheet("FullList")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Schritt 5: Filtertabelle "FilterRoles" einlesen
worksheet_roles = spreadsheet.worksheet("FilterRoles")
df_roles = get_as_dataframe(worksheet_roles, evaluate_formulas=True)

# Schritt 6: Nur relevante Rollen (wo 'Role Relevant' TRUE ist)
relevant_roles = df_roles[
    df_roles['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE'
]['Job Role'].dropna().astype(str).str.strip().tolist()

# Optional: Vorschau
print("Datenvorschau:")
display(df.head())

# Schritt 7: Relevante Spalten vorbereiten
df_relevant = df[['HOME FUNCTION', 'Job Role']].copy()
df_relevant.columns = ['FunctionGroup', 'JobRole']
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')

# Schritt 8: JobRoles aufsplitten und jede in eigene Zeile bringen
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# Schritt 9: Nur relevante Rollen behalten
df_filtered = df_exploded[df_exploded['JobRole'].isin(relevant_roles)].copy()

# Schritt 10: Gruppieren und Pivot-Tabelle erzeugen
df_grouped = df_filtered.groupby(['FunctionGroup', 'JobRole']).size().reset_index(name='Count')
df_pivot = df_grouped.pivot(index='FunctionGroup', columns='JobRole', values='Count').fillna(0).astype(int)

# Optional: Sortieren
df_pivot = df_pivot.sort_index()
df_pivot = df_pivot.reindex(sorted(df_pivot.columns), axis=1)

# Ergebnis anzeigen
print("Pivot-Tabelle nach HOME FUNCTION mit gefilterten Job Roles:")
display(df_pivot)

# Exportieren als CSV
df_pivot.to_csv("job_role_summary_by_function.csv")
print("CSV gespeichert als 'job_role_summary_by_function.csv'")


## Zusammenfassung des Notebooks

Dieses Notebook führt eine detaillierte Analyse von Jobrollen und deren Verteilung innerhalb verschiedener Funktionseinheiten (HOME FUNCTION) durch. Es verwendet Daten aus zwei Google Sheets: "FullList" (mit Personen- und Rolleninformationen) und "FilterRoles" (zur Definition relevanter Jobrollen).

Hier sind die Hauptschritte und Ergebnisse:

1.  **Datenvorbereitung und Filterung:**
    *   Es werden Daten aus den Google Sheets "FullList" und "FilterRoles" geladen.
    *   Nur Jobrollen, die in "FilterRoles" als 'Relevant' markiert sind, werden für die Analyse berücksichtigt.
    *   Mehrere Jobrollen pro Person werden aufgeteilt, sodass jede Rolle einzeln analysiert werden kann.

2.  **Rollenverteilung nach 'HOME FUNCTION' (Pivot-Tabellen):**
    *   Es werden drei verschiedene Übersichten zur Verteilung von Jobrollen erstellt:
        *   Eine Pivot-Tabelle, die die Anzahl der Personen pro 'HOME FUNCTION' und 'Job Role' anzeigt (exportiert als `job_role_summary_by_function.csv`).
        *   Eine gruppierte Liste der Kombinationen aus 'HOME FUNCTION' und 'Job Role' mit der jeweiligen Anzahl der Personen (exportiert als `job_role_summary_airtable_format.csv`).
        *   Eine Pivot-Tabelle, die die Anzahl der Personen pro 'Job Role' und 'HOME FUNCTION' darstellt, wobei auch Kombinationen mit 0 Personen angezeigt werden (exportiert als `job_role_function_matrix.csv`).

3.  **Personenbezogene Rollenaggregation:**
    *   Es wird eine Liste generiert, die jede Person mit ihren zugeordneten relevanten Jobrollen (als kommagetrennte Liste) anzeigt.
    *   Diese Liste ist nach 'HOME FUNCTION' und dann nach 'EMPLOYEE FULL NAME' alphabetisch sortiert (exportiert als `person_role_filtered_aggregated.csv`).

4.  **Spezifische Analyse der Rolle "1201_General Assets Display":**
    *   Es wird analysiert, wie viele Personen **nur** die Rolle "1201_General Assets Display" haben, im Vergleich zu denen, die diese Rolle **plus** mindestens eine weitere relevante Rolle besitzen.
    *   Diese Verteilung wird in einem Kuchendiagramm visualisiert.

5.  **Verteilung der Anzahl relevanter Rollen pro Person:**
    *   Ein Histogramm zeigt die Häufigkeit, mit der Personen eine bestimmte Anzahl relevanter Rollen haben (bis zu 30 Rollen).
    *   Personen mit mehr als 30 relevanten Rollen werden separat tabellarisch aufgeführt und zusammengefasst.

Das Notebook bietet somit eine umfassende Übersicht über die Verteilung von Jobrollen, sowohl auf aggregierter Ebene nach Funktionseinheit als auch detailliert pro Person, und beinhaltet spezifische Analysen für bestimmte Rollenkonfigurationen.

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren für Google Drive & Sheets
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen über URL
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Reiter "FullList" einlesen als DataFrame
worksheet = spreadsheet.worksheet("FullList")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Schritt 5: Filtertabelle "FilterRoles" einlesen
worksheet_roles = spreadsheet.worksheet("FilterRoles")
df_roles = get_as_dataframe(worksheet_roles, evaluate_formulas=True)

# Schritt 6: Nur relevante Rollen (wo 'Role Relevant' TRUE ist)
relevant_roles = df_roles[
    df_roles['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE'
]['Job Role'].dropna().astype(str).str.strip().tolist()

# Schritt 7: Relevante Spalten vorbereiten
df_relevant = df[['HOME FUNCTION', 'Job Role']].copy()
df_relevant.columns = ['FunctionGroup', 'JobRole']
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')

# Schritt 8: JobRoles aufsplitten und jede in eigene Zeile bringen
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# Schritt 9: Nur relevante Rollen behalten
df_filtered = df_exploded[df_exploded['JobRole'].isin(relevant_roles)].copy()

# Schritt 10: Gruppieren und Zählung
df_grouped = (
    df_filtered
    .groupby(['FunctionGroup', 'JobRole'])
    .size()
    .reset_index(name='Number of Roles in ORganization')
)

# Nur Kombinationen mit mindestens einer Person behalten
df_grouped = df_grouped[df_grouped['Number of Roles in ORganization'] > 0]

# Schritt 11: Ergebnis anzeigen
print("Tabellenformat wie in AirtableAssetsRoles - Sheet8.csv:")
display(df_grouped)


# Schritt 12: Exportieren als CSV
df_grouped.to_csv("job_role_summary_airtable_format.csv", index=False)
print("CSV gespeichert als 'job_role_summary_airtable_format.csv'")


geb mir einen weiteren code der es in das format wie in der csv vorgegeben. jetzt muss allerdings auch wieder "0" angezeigt weredn

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Daten aus "FullList" holen
worksheet = spreadsheet.worksheet("FullList")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Schritt 5: Filterrollen aus "FilterRoles" holen
worksheet_roles = spreadsheet.worksheet("FilterRoles")
df_roles = get_as_dataframe(worksheet_roles, evaluate_formulas=True)

# Nur Rollen mit TRUE in 'Role Relevant'
relevant_roles = df_roles[
    df_roles['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE'
]['Job Role'].dropna().astype(str).str.strip().tolist()

# Schritt 6: Relevante Spalten vorbereiten
df_relevant = df[['HOME FUNCTION', 'Job Role']].copy()
df_relevant.columns = ['FunctionGroup', 'JobRole']
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')

# Schritt 7: Explodieren und Filtern
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()
df_filtered = df_exploded[df_exploded['JobRole'].isin(relevant_roles)].copy()

# Schritt 8: Zählen (inkl. 0er durch Pivot mit fillna)
df_counts = (
    df_filtered
    .groupby(['JobRole', 'FunctionGroup'])
    .size()
    .reset_index(name='Count')
)

# Alle möglichen Kombinationen generieren (auch mit 0en)
all_roles = pd.Series(relevant_roles, name='JobRole').drop_duplicates()
all_functions = df_filtered['FunctionGroup'].dropna().unique()

multi_index = pd.MultiIndex.from_product([all_roles, all_functions], names=['JobRole', 'FunctionGroup'])
df_complete = pd.DataFrame(index=multi_index).reset_index()
df_complete = df_complete.merge(df_counts, on=['JobRole', 'FunctionGroup'], how='left')
df_complete['Count'] = df_complete['Count'].fillna(0).astype(int)

# Schritt 9: Pivotieren ins Zielformat
df_pivot = df_complete.pivot(index='JobRole', columns='FunctionGroup', values='Count').fillna(0).astype(int)
df_pivot.reset_index(inplace=True)

# Schritt 10: Ergebnis anzeigen
print("Tabelle im Format von Sheet9 (JobRole × FunctionGroup mit 0-Werten):")
display(df_pivot)

# Schritt 11: Exportieren
df_pivot.to_csv("job_role_function_matrix.csv", index=False)
print("CSV gespeichert als 'job_role_function_matrix.csv'")


ich brauche noch eine Auswertung, welche mir die Personen auflistet und ihre Rollen, wenn die "FilterRole" "TRUE" sind. Vorangestellt geordnet nach "HOME FUNCTION". Alphabetisch sortiert die Namen. Tabelle nach dem csv Muster.

Liest den Reiter "FullList" ein.

Berücksichtigt nur "Job Roles" mit "TRUE" aus "FilterRoles".

Trennt mehrere Rollen pro Person auf.

Gibt eine Zeile pro Person und Rolle aus.

Sortiert nach:

HOME FUNCTION (aufsteigend),

EMPLOYEE FULL NAME (alphabetisch).

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Daten aus "FullList" einlesen
worksheet = spreadsheet.worksheet("FullList")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Schritt 5: Relevante Rollen aus "FilterRoles"
worksheet_roles = spreadsheet.worksheet("FilterRoles")
df_roles = get_as_dataframe(worksheet_roles, evaluate_formulas=True)
relevant_roles = df_roles[
    df_roles['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE'
]['Job Role'].dropna().astype(str).str.strip().tolist()

# Schritt 6: Spalten bereinigen und Job Roles splitten
df['Job Role'] = df['Job Role'].fillna('').astype(str).str.split(',')
df_exploded = df.explode('Job Role')
df_exploded['Job Role'] = df_exploded['Job Role'].str.strip()

# Schritt 7: Nur relevante Rollen filtern
df_filtered = df_exploded[df_exploded['Job Role'].isin(relevant_roles)].copy()

# Schritt 8: Gruppieren auf Mitarbeiterebene, Rollen aggregieren
columns_to_keep = [
    'HOME FUNCTION', 'EMPLOYEE FULL NAME', 'EMPLOYEE EMAIL', 'EMPLOYEE ID',
    'EMPLOYEE POSITION', 'EMPLOYEE LOCATION', 'PEOPLE MANAGER',
    '# of mapped Roles', 'Mapping Status'
]

# Entferne Duplikate (1 Zeile pro Person, Rollen werden gesammelt)
df_unique = df_filtered[columns_to_keep].drop_duplicates()

# Aggregierte Rollenliste (kommagetrennt)
df_roles_combined = (
    df_filtered.groupby('EMPLOYEE EMAIL')['Job Role']
    .apply(lambda x: ', '.join(sorted(set(x))))
    .reset_index()
)

# Merge mit den restlichen Daten
df_final = pd.merge(df_unique, df_roles_combined, on='EMPLOYEE EMAIL', how='left')

# Schritt 9: Sortieren
df_final = df_final.sort_values(by=['HOME FUNCTION', 'EMPLOYEE FULL NAME'])

# Schritt 10: Ergebnis anzeigen
print("Gefilterte Liste (eine Zeile pro Person, Rollen kommagetrennt):")
display(df_final)

# Schritt 11: Exportieren
df_final.to_csv("person_role_filtered_aggregated.csv", index=False)
print("CSV gespeichert als 'person_role_filtered_aggregated.csv'")

Liest Reiter Person_Role und FilterRoles.

Zählt, wie viele Personen:

nur die Rolle 1201_General Assets Display haben,

mindestens eine andere relevante Rolle zusätzlich haben.

Schreibt das Ergebnis in einen neuen Reiter Asset_Display_Only_Stats.

Zeigt ein Balkendiagramm in Colab an mit matplotlib.



In [ ]:
# Schritt 1: Bibliotheken installieren
!pip install --quiet gspread pandas gspread-dataframe google-auth matplotlib

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
import google.auth
import matplotlib.pyplot as plt

# Schritt 2: Authentifizieren
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Tabelle öffnen
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Daten aus "Person_Role" und "FilterRoles" laden
ws_roles = spreadsheet.worksheet("Person_Role")
df_roles = get_as_dataframe(ws_roles, evaluate_formulas=True)

ws_filter = spreadsheet.worksheet("FilterRoles")
df_filter = get_as_dataframe(ws_filter, evaluate_formulas=True)

# Schritt 5: Relevante Rollen extrahieren
relevant_roles = df_filter[
    df_filter['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE'
]['Job Role'].dropna().astype(str).str.strip().tolist()

# Schritt 6: Rollenliste je Person bereinigen
df_roles = df_roles[['EMPLOYEE FULL NAME', 'EMPLOYEE EMAIL', 'HOME FUNCTION', 'Job Role']].copy()
df_roles['Job Role'] = df_roles['Job Role'].fillna('').astype(str).str.split(',')

# Relevante Rollen extrahieren je Person
df_roles['Job Role'] = df_roles['Job Role'].apply(
    lambda lst: [r.strip() for r in lst if r.strip() in relevant_roles]
)

# Prüfen, ob nur genau "1201_General Assets Display" vorkommt
df_roles['Only_Display'] = df_roles['Job Role'].apply(
    lambda roles: len(roles) == 1 and roles[0] == "1201_General Assets Display"
)

# Schritt 7: Zählung
total_persons = len(df_roles)
only_display = df_roles['Only_Display'].sum()
with_others = total_persons - only_display

# Zusammenfassen
summary_df = pd.DataFrame({
    'Kategorie': ['Nur "1201_General Assets Display"', 'mind. eine weitere Asset Rolle enthalten'],
    'Anzahl Personen': [only_display, with_others]
})


Erstellt Kuchendiagramm


In [ ]:
labels = summary_df['Kategorie']
sizes = summary_df['Anzahl Personen']
colors = ['lightgreen', 'lightblue']
total = sum(sizes)

# Prozent + Anzahl kombinieren
def make_autopct(values):
    def my_autopct(pct):
        count = int(round(pct * total / 100.0))
        return f'{pct:.1f}%\n({count})'
    return my_autopct

plt.figure(figsize=(7, 7))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=labels,
    autopct=make_autopct(sizes),
    colors=colors,
    startangle=140,
    textprops={'fontsize': 12}
)

# Titel und Kreisform
plt.title("Verteilung: Nur '1201_General Assets Display' vs. weitere Rollen", fontsize=12, pad=10)
plt.axis('equal')

# ➕ Gesamtanzahl unterhalb anzeigen
plt.figtext(0.5, 0.05, f"Gesamtanzahl der Personen: {total}", ha='center', fontsize=10, fontweight='bold')

#plt.tight_layout()
plt.show()


5

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Nur Personen mit mindestens 1 relevanter Rolle
rollen_counts = df_roles[df_roles['Anzahl Rollen'] > 0]['Anzahl Rollen']

# Werte aufteilen in zwei Gruppen
rolle_max = 30
rollen_bis_30 = rollen_counts[rollen_counts <= rolle_max]
rollen_ueber_30 = rollen_counts[rollen_counts > rolle_max]

# Verteilung > 30 tabellarisch zählen
ueber_30_verteilung = rollen_ueber_30.value_counts().sort_index()
df_ueber_30 = pd.DataFrame({
    'Anzahl Rollen': ueber_30_verteilung.index,
    'Anzahl Personen': ueber_30_verteilung.values
})

# Histogramm 1–30
bins = range(1, rolle_max + 2)  # 1 bis 31
plt.figure(figsize=(10, 5))
counts, edges, bars = plt.hist(
    rollen_bis_30,
    bins=bins,
    edgecolor='black',
    color='skyblue',
    align='left'
)

# Beschriftung & Layout
plt.xlabel("Anzahl relevanter Rollen pro Person (1–30)", fontsize=11)
plt.ylabel("Anzahl Personen", fontsize=11)
plt.title("Histogramm: Rollenanzahl pro Person (ab 1, max 30)", fontsize=13)
plt.xticks(bins, fontsize=9)
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Beschriftung über Balken (senkrecht & klein)
for bar in bars:
    height = bar.get_height()
    if height > 0:
        plt.text(
            bar.get_x() + bar.get_width()/2,
            height + 5,
            f"{int(height)}",
            ha='center',
            va='bottom',
            fontsize=8,
            rotation=90
        )

plt.tight_layout()
plt.show()

# Tabellarische Anzeige für > 30 Rollen
print("🔢 Personen mit mehr als 30 Rollen (nicht im Diagramm):")
display(df_ueber_30)


In [ ]:

# Zusammenfassung
gesamt_ueber_30 = df_ueber_30['Anzahl Personen'].sum()
print(f"👉 Insgesamt haben {gesamt_ueber_30} Personen mehr als 30 relevante Rollen.")
